In [6]:
from preprocessing import preprocessamento
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings('ignore')

pre = preprocessamento.Preprocessamento()

df1, df2, df3, df4, df5 = map(pre.ler_csv, ['imob1.csv', 'imob2.csv',
                                            'imob3.csv', 'imob4.csv',
                                            'imob5.csv'])

lista = [df1, df2, df3, df4, df5]

df3['banheiros'] = df3['suites']
df4['banheiros'] = df4['suites']

for l in lista:
    l.drop(columns='suites', inplace=True)

dataset = pre.concatenar_datasets([df1, df2, df3, df4, df5])
dataset = dataset.iloc[:, 2:]
dataset = pre.ajustar_texto(dataset)
dataset = pre.ajustar_nomes_bairros(dataset)
coordenadas = pre.ler_csv('bairros_e_coordenadas.csv')
dataset = pre.inserir_coordenadas_geograficas(dataset, coordenadas)

"""Recomendador e busca"""

# Retira linhas que a coluna valor == 0
# e reseta o index
mask = dataset['valor'] != 0
dataset = dataset[mask]


def reset_dataset_index(dataset):
    dataset.reset_index(inplace=True)
    dataset.drop(columns='index', inplace=True)
    return dataset


dataset = reset_dataset_index(dataset)


def knn(vizinhos, X, features):
    neigh = NearestNeighbors(n_neighbors=vizinhos, metric='euclidean').fit(X)
    distance, indices = neigh.kneighbors([features])
    #print(distance)
    return list(indices[0]), list(distance[0])


def coordenadas_geograficas(bairro):
    mask = coordenadas['BAIRROS'] == bairro
    return list(coordenadas.loc[mask, ['LATITUDE', 'LONGITUDE']].values[0])


def bairro_igual_ou_diferente(bairro, opcao):
    if opcao == 0:
        mask = dataset['bairro'] != bairro
    if opcao == 1:
        mask = dataset['bairro'] == bairro
    data = dataset.loc[mask]
    data = reset_dataset_index(data)
    return data, data.iloc[:, 1:].values


def apartamentos(indices, distancia, dataset):
    for i, d in zip(indices, distancia):
        apto = dataset.iloc[i]
        print('Bairro {} com {} quarto(s), {} banheiro(s), {} vaga(s) de garagem, área de {}m² e valor de R$ {}.'.format(
            apto['bairro'], apto['quartos'], apto['banheiros'], apto['garagens'], apto['area'], round(apto['valor'], 2)))
        print('Dissimilaridade: {}'.format(round(d, 4)))
        print('\n')
        

def verificar_samples(dataset, bairro):
    return dataset.loc[dataset['bairro'] == bairro, 'bairro'].count()


def retorna_indices(dataset, bairro):
    return list(dataset.loc[dataset['bairro'] == bairro, 'bairro'].index)


def buscar(bairro, quartos, banheiros, garagens, area, valor):
    vizinhos = 5
    data, x = bairro_igual_ou_diferente(bairro, 1)
    quantidade = verificar_samples(data, bairro)
    if quantidade == 0:
        print('Não foram encontrados apartamentos neste bairro.')
        return 0
    elif quantidade < vizinhos:
        indices = retorna_indices(data, bairro)
        print('Apartamentos encontrados: ')
        apartamentos(indices, data)
        return 0
    else:
        pontos = coordenadas_geograficas(bairro)
        features = pontos + [quartos, banheiros, garagens, area, valor]
        indices = knn(vizinhos, x, features)
        print('Apartamentos encontrados: ')
        apartamentos(indices, data)


def recomendar(bairro, quartos, banheiros, garagens, area, valor):
    vizinhos = 20
    data, x = bairro_igual_ou_diferente(bairro, 0)
    pontos = coordenadas_geograficas(bairro)
    features = pontos + [quartos, banheiros, garagens, area, valor]
    indices, distancia = knn(vizinhos, x, features)
    print('Imóveis recomendados: \n')
    apartamentos(indices, distancia, data)

In [8]:
recomendar('Centro', 3, 2, 1, 70, 200000)

Imóveis recomendados: 

Bairro Jardim Panorama com 2 quarto(s), 1 banheiro(s), 1 vaga(s) de garagem, área de 75.53m² e valor de R$ 200000.0.
Dissimilaridade: 5.708


Bairro Todos Os Santos com 3 quarto(s), 2 banheiro(s), 1 vaga(s) de garagem, área de 0.0m² e valor de R$ 200000.0.
Dissimilaridade: 70.0


Bairro Canelas I com 3 quarto(s), 1 banheiro(s), 1 vaga(s) de garagem, área de 0.0m² e valor de R$ 200000.0.
Dissimilaridade: 70.0071


Bairro Morada Do Parque com 3 quarto(s), 1 banheiro(s), 1 vaga(s) de garagem, área de 0.0m² e valor de R$ 200000.0.
Dissimilaridade: 70.0072


Bairro Canelas com 2 quarto(s), 1 banheiro(s), 1 vaga(s) de garagem, área de 0.0m² e valor de R$ 200000.0.
Dissimilaridade: 70.0143


Bairro Ibituruna com 2 quarto(s), 1 banheiro(s), 1 vaga(s) de garagem, área de 0.0m² e valor de R$ 200000.0.
Dissimilaridade: 70.0143


Bairro Ibituruna com 2 quarto(s), 0 banheiro(s), 1 vaga(s) de garagem, área de 0.0m² e valor de R$ 200000.0.
Dissimilaridade: 70.0357


Bairro Aug